In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
INPUT = "final_with_indexes.csv"
OUT_PREFIX = "forecast_2030_"

df = pd.read_csv(INPUT)

COL_YEAR = "Year"
COL_ISO  = "ISO3"

TARGETS = {
    "GDP growth (annual %)": "gdp_growth_2030",
    "Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)": "poverty_2030",
    "ResilienceScore": "resilience_2030",
}

candidate_features = [
    "TDI",
    "ResilienceScore",
    "SpendingEfficiency",
    "ShockImpact",
    "Trade (% of GDP)",
    "Exports of goods and services (% of GDP)",
    "Imports of goods and services (% of GDP)",
    "GDP growth (annual %)",
    "Inflation, consumer prices (annual %)",
    "Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)",
    "Population_growth_(annual_%)",
    "Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)",
    "Total Damage, Adjusted ('000 US$)",
]

features = [c for c in candidate_features if c in df.columns]

df = df.dropna(subset=[COL_ISO, COL_YEAR]).copy()
df[COL_YEAR] = df[COL_YEAR].astype(int)

if "GDP (current US$)" in df.columns and "Total_Population_Both_sexes" in df.columns:
    df["GDPpc"] = df["GDP (current US$)"] / df["Total_Population_Both_sexes"].replace(0, np.nan)
    features.append("GDPpc")

features = [f for f in sorted(set(features)) if f in df.columns]


last_year = int(df[COL_YEAR].max())
valid_years = list(range(last_year - 1, last_year + 1))
train_df = df[~df[COL_YEAR].isin(valid_years)].copy()
valid_df = df[df[COL_YEAR].isin(valid_years)].copy()

def train_one_target(target_col):
    trn = train_df.dropna(subset=[target_col] + features).copy()
    vld = valid_df.dropna(subset=[target_col] + features).copy()

    if trn.empty or len(trn[COL_ISO].unique()) < 5:
        print(f"[WARN] Not enough training data for target: {target_col}. Skipping validation.")
    X_tr, y_tr = trn[features], trn[target_col]
    X_vl, y_vl = vld[features], vld[target_col] if not vld.empty else (None, None)

    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=True, with_std=True), list(range(len(features)))),
        ],
        remainder="drop"
    )

    model = RandomForestRegressor(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )

    pipe = Pipeline(steps=[("scale", pre), ("rf", model)])
    pipe.fit(X_tr, y_tr)

    if X_vl is not None and not vld.empty:
        y_hat = pipe.predict(X_vl)
        print(f"[{target_col}]  R2(valid)={r2_score(y_vl, y_hat):.3f}  MAE(valid)={mean_absolute_error(y_vl, y_hat):.3f}")
    else:
        print(f"[{target_col}]  Validation skipped (no data).")

    return pipe

models = {}
for t in TARGETS.keys():
    if t in df.columns:
        models[t] = train_one_target(t)
    else:
        print(f"[WARN] Target column missing: {t}")

latest = (
    df.sort_values([COL_ISO, COL_YEAR])
      .groupby(COL_ISO, as_index=False)
      .tail(1)
      .reset_index(drop=True)
)

for col in features:
    if latest[col].isna().any():
        med = df.groupby(COL_ISO)[col].transform('median')
        latest[col] = latest[col].fillna(med)
        latest[col] = latest[col].fillna(df[col].median())

latest[COL_YEAR] = 2030

def clip_pct(series, low=-50, high=200):
    return series.clip(lower=low, upper=high)


def scenario_baseline(base):
    s = base.copy()
    return s

def scenario_social_spending(base):
    s = base.copy()
    if "Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)" in s.columns:
        s["Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)"] = \
            (s["Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)"] * 0.92).clip(lower=0)  # -8%
    if "Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)" in s.columns:
        s["Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)"] = \
            (s["Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)"] * 0.95).clip(lower=0)  # -5%
    if "GDP growth (annual %)" in s.columns:
        s["GDP growth (annual %)"] = clip_pct(s["GDP growth (annual %)"] + 0.6)  # +0.6 pp
    return s

def scenario_trade_diversification(base):
    s = base.copy()
    if "TDI" in s.columns:
        s["TDI"] = (s["TDI"] * 0.8).clip(lower=0)
    if "Trade (% of GDP)" in s.columns:
        s["Trade (% of GDP)"] = clip_pct(s["Trade (% of GDP)"] * 1.03)  # +3%
    return s

def scenario_global_crisis(base):
    s = base.copy()
    if "Total Damage, Adjusted ('000 US$)" in s.columns:
        s["Total Damage, Adjusted ('000 US$)"] = s["Total Damage, Adjusted ('000 US$)"] * 1.5  # +50%
    if "Trade (% of GDP)" in s.columns:
        s["Trade (% of GDP)"] = clip_pct(s["Trade (% of GDP)"] * 0.85)  # -15%
    if "GDP growth (annual %)" in s.columns:
        s["GDP growth (annual %)"] = clip_pct(s["GDP growth (annual %)"] - 2.0)  # -2 pp
    if "Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)" in s.columns:
        s["Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)"] = \
            (s["Unemployment,_total_(%_of_total_labor_force)_(modeled_ILO_estimate)"] * 1.08).clip(lower=0)  # +8%
    if "TDI" in s.columns:
        s["TDI"] = (s["TDI"] * 1.05).clip(upper=1.0)
    return s

SCENARIOS = {
    "baseline": scenario_baseline,
    "social_spending": scenario_social_spending,
    "trade_diversification": scenario_trade_diversification,
    "global_crisis": scenario_global_crisis,
}


def predict_for_scenario(name, base_latest):
    scen_df = SCENARIOS[name](base_latest)

    out = scen_df[[COL_ISO, COL_YEAR]].copy()
    for target_col, out_name in TARGETS.items():
        if target_col in models:
            X = scen_df[features].copy()
            for col in features:
                if X[col].isna().any():
                    X[col] = X[col].fillna(df[col].median())
            preds = models[target_col].predict(X)
            out[out_name] = preds
        else:
            out[out_name] = np.nan
    return out

all_results = []
for scen in SCENARIOS:
    pred = predict_for_scenario(scen, latest)
    pred["scenario"] = scen
    all_results.append(pred)

final_preds = pd.concat(all_results, ignore_index=True)

for scen in SCENARIOS:
    scen_df = final_preds[final_preds["scenario"] == scen].copy()
    scen_df.to_csv(f"{OUT_PREFIX}{scen}.csv", index=False)

final_preds.to_csv(f"{OUT_PREFIX}ALL.csv", index=False)
for scen in SCENARIOS:
    print(f" - {OUT_PREFIX}{scen}.csv")
print(f" - {OUT_PREFIX}ALL.csv")




[GDP growth (annual %)]  R2(valid)=0.999  MAE(valid)=0.000
[Poverty_headcount_ratio_at_$3.00_a_day_(2021_PPP)_(%_of_population)]  R2(valid)=1.000  MAE(valid)=0.022
[ResilienceScore]  R2(valid)=1.000  MAE(valid)=0.006
 - forecast_2030_baseline.csv
 - forecast_2030_social_spending.csv
 - forecast_2030_trade_diversification.csv
 - forecast_2030_global_crisis.csv
 - forecast_2030_ALL.csv
